# pandas lesson 

## Introduction

pandas is *the* library for data analysis in Python.  It has two data structures: 
* the Series for 1D labelled data such as a single row or column,
* the DataFrame for 2D data such as a table. 

A good place to get started with pandas is at https://pandas.pydata.org/getting_started.html

 This lesson shows examples of typical operations on a pandas DataFrame including:
* select a subset of columns
* calculate new columns
* filter rows in various ways
* group and summarise
* handle missing values


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # pandas uses matplotlib for plotting

## pandas Series (a 1D labelled array)

A Series is a 1D labelled array.  By default the labels are position-based integers, starting at 0.  Labels don't need to be unique.  The elements of a Series are usually of the same type. A Series may become a column in a dataframe (table) so we should expect this. These types include various types of number (ints and floats) and strings (objects).

We can create a Series in many ways, for example from a list.

### Create a Series

In [ ]:
# Create a Series form a list of first names
names = ['Harry', 'Hermione', 'Ron']
first_names = pd.Series(names)
first_names

In [ ]:
#  We can pass in an index when creating a Series
initials = ['hp', 'hg', 'rw']
first_names = pd.Series(names, index=initials)
first_names

Examine the first_names Series.  You may want to try these properties and methods: index, values, dtype, shape, ndim, size.

In [ ]:
# Write your code here as a set of print statements. The first one is provided.
print("Index:", first_names.index) 

We can create a Series with a defined size and initialize with fixed or random values.

In [ ]:
# a Series of 4 random numbers with mean 10 and standard deviation 1
pd.Series(np.random.randn(4), name = 'price') + 10 # name is an optional parameter

### Access elements in a Series

We can access elements of the Series 
* by position using the iloc property,  or 
* by their index using the loc property. 

In [ ]:
# returns the item in the 2nd position
first_names.iloc[1]

In [ ]:
# returns the item in the 2nd and 3rd positions
first_names.iloc[1:3] 

In [ ]:
#  Returns the element but using the index label
first_names.loc['rw']

Note that when slicing with loc, the syntax is inclusive (and not the usual Pythton syntax!).

In [ ]:
first_names.loc['hp':'rw']

We can use the index to set values from the Series.

In [ ]:
print("before change:", first_names.loc['rw']) 
first_names['rw'] = 'Ronald' # set a value in the Series
print("after change:", first_names.loc['rw']) 

We can use *in* to see if the index value is in the Series

In [ ]:
'hg' in first_names # check if an index is in the Series

### Element wise operations

An element wise operation is one that is performed on every element of the Series. For example,  multiply all values by 100

The examples in this section use a Series of 5 numbers named prices that is defined below.

In [ ]:
prices = pd.Series([10, 12, 15, 18, 16])
prices

THis multiplies every elemnt by a scalar value

In [ ]:
prices * 100

Add 10 to each value in the prices Series.

In [ ]:
# Write your code here

We can aggregate (e.g sum. average) the values in a Series either 
* using a numpy method, e.g. np.sum(prices)
* a method on the Series e.g. prices.sum()

In [ ]:
np.sum(prices) # total value of all elements

In [ ]:
prices.sum() 

Find the min, max, average, median and other summary statistics of the prices Series

In [ ]:
# Write your code here as a set of print statements. The first one is provided.
print(f"minimum: numpy method {np.min(prices)}, Series method {prices.min()}")



## pandas DataFrames

A DataFrame is basically a table.  It is a 2D labelled data structure.  The columns can be different types.  You can think of it as a dict of Series objects (columns) if that helps.  Like a Series, a Dataframe has an index column.

We can build a pandas Dataframe in many ways, for example from a dict. The dict's keys become the column names and the dict's values become the column values.

We can use the pandas read_csv method to load data directly into a DataFrame.  WE will use the epl DataFrame in the following sections.This contains the football English Premier League results from teh 2023-24 season as provided by ChatGPT!)

In [ ]:
epl_csv_file_url = "https://zomalextrainingstorage.blob.core.windows.net/datasets/misc/EPL%20Results%202023-24.csv"
epl = pd.read_csv(epl_csv_file_url)
epl.head()

Use the three letter code for the team as the index

In [ ]:
epl.set_index('Code', inplace=True)
epl.head()

Exercise: Examine the dataframe.  
Use the following dataframe properties and methods: index, head(), describe(), shape, values, columns.

In [ ]:
# Write your code here as a set of print statements. The first one is provided.
print("Index:\n", epl.index)

### Access columns in a Dataframe

We can access a columns or columns, col1, col2 in a DataFrame df in several ways: 
* df['col1'] which returns a series
* df.col1 which returns a series (pandas allows us to refer to column names like a property! )
* df[['col1']] or df[['col1', 'col2']] which return DataFrames
* df.loc[ : , ['col1', 'col2']] which return DataFrames and is possibly best practice and most flexible

Task: experiment with various ways of selecting columns of the epl DataFrame.

In [ ]:
# Here are some examples
print("Using dot notation:\n", epl.Team.head(2))
print("Using [] slice notation:\n", epl['Team'].head(2))
# Write your code here


### Sort rows
We can sort by values or sort by index.  Sort by values is the more usual case.  

Order the rows in the dataframe by the number of games won (low to high), then in the case of any ties, by the number of games drawn.

In [ ]:
epl.sort_values(by = ['Won', 'Drawn'], ascending=True)

Task: sort rows by the GF and GA column (highest first)

In [ ]:
# Write your code here

We can also sort a DataFrame by its index.

In [ ]:
epl_sorted_by_index = epl.sort_index()
epl_sorted_by_index.head()

### Create new columns

In [ ]:
# Teams get 3 point for a win, 1 for a draw, none for a loss
epl['Points'] = epl['Won'] * 3 + epl['Drawn']
epl.head()

Task: Create a new column, Played, to hold the number of games played by each team.


In [ ]:
# Write your code here

Task: Create a new column, GD, to hold the goal difference, calculated as GF - GA (Goals For - Goals Against)

In [ ]:
# Write your code here

### Filter rows
Filter both rows and columns of the dataframe in various ways using the loc method and the index / column names.

In [ ]:
epl.loc['LIV', :] # one row, all columns, returns a Series

In [ ]:
epl.loc[['LIV'], :] # one row, all columns, returns a DataFrame

In [ ]:
epl.loc[['LIV', 'MCY'], :] #   two rows, all columns, returns a DataFrame

In [ ]:
epl.loc[['LIV', 'MCY'], ['Team', 'Won']] # two rows, two columns

### Filter Rows by Values

We have already seen that a boolean expression returns a Series of bools

In [ ]:
epl.Won >= 25


We can use this boolean expression to filter the rows of the DataFrame where the bools are True

In [ ]:
epl.loc[epl.Won >= 25, :]

Task: filter the epl DataFrame to return only teams with 80 points or more

In [ ]:
# Write your code here


Filter teams that have Manchester in their name

In [ ]:

epl.loc[epl.Team.str.contains('Manchester'), :]

We can combine two filter criteria with an & (and)

In [ ]:
epl.Team.str.contains('Manchester') & (epl.Won > 25)

Task:  filter the epl dataframe to return only London teams that have won more than 5 games

In [ ]:
# Write your code here

### Group data

In this section we will use a tiny DataFrame based on the fb dict below

In [ ]:
fb_dict = {
        'id': ['MCY', 'LIV', 'TOT', 'CHE', 'ARL'],
        'city': ['Manchester',	'Liverpool', 'London', 'London', 'London'],
        'team':	['Manchester City', 'Liverpool', 'Tottenham Hotspur', 'Chelsea', 'Arsenal'],
        'champions_league': ['Yes', 'Yes', 'No', 'No', 'Yes'],
        'won':	[5, 6, 6, 5, 5],
        'drawn': [4, 1, 0, 2,0],
        'lost': [0, 0, 2, 0, 2],
        'form': ['DWWWW', 'WWWWD', 'LLWWW', 'WWWDD', 'WWWWW']
        }

fb = pd.DataFrame(fb_dict)
fb


In [ ]:
# set the index to the unique values of the 'id' column - more useful than 0,1,2...
fb = fb.set_index('id')
fb

Group by city

In [ ]:
fb_by_city = fb.groupby(['city'], as_index=False)
fb_by_city.groups

Get the totals of all the numeric columns per city

In [ ]:
fb_by_city.sum(numeric_only=True)
fb_by_city[['points', 'won']].sum()


Exercise: group by those teams in (and not in) the Champions League (champions_league = 'Yes' or 'No')
Sum the won, drawn and lost  columms

In [ ]:
# Write your code here

In [ ]:
fb.loc[:, ['city']] # all rows, one column

In [ ]:
fb.loc[:, ['team', 'won']] # all rows, two columns